In [ ]:
import os
import sys
sys.path.append(os.path.abspath('../'))
import pandas as pd
from pathlib import Path

from source.naver_geocode import NAVERGeocoding

from tqdm import tqdm

datapath = "localdata_gov_2022_08_07.csv"
save_path = "./gov_geocoded.csv"


In [ ]:
data = pd.read_csv(datapath, encoding="CP949", sep=",")
data = data.loc[data["영업상태구분코드"].isin([1])]  #  01: 영업/정상, 02:휴업, 03: 폐업, 04: 취소/말소/만료/정지/중지
data.sort_values(by=["데이터갱신일자"], ascending=False, inplace=True)

data = data[["사업장명", "도로명전체주소", "소재지전화"]]
data.rename(columns={"사업장명":"name","도로명전체주소": "addr", "소재지전화": "phone"}, inplace=True)
data.fillna("", inplace=True)


In [ ]:

geocoding = NAVERGeocoding(secret_key_path="../../sources/json/secret.json", geocode_url="https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode")

if 'NAVER_addr' not in data.columns:
    data = data.assign(x="")
    data = data.assign(y="")
    data = data.assign(NAVER_addr="")
else:
    pass

data.fillna("", inplace=True)
for idx, row in tqdm(data.iterrows()):
    if row["x"] and row["y"] and row["NAVER_addr"] :
        continue  # fill empty cases only
    try:
        addr = row["addr"]
        x, y, naver_addr = geocoding(addr)
        data.loc[idx, ["x", "y", "NAVER_addr"]] = x, y, naver_addr
    except Exception as e:  # 예외 발생한 아웃라이어에 대해서는 직접 수정 필요
        print(idx, e)
        pass
data.to_csv(save_path,sep=",", encoding="UTF-8", header=True, index=False)